In [46]:
import torch
import matplotlib.pyplot as plt
%matplotlib inline
import numpy
import torch.nn.functional as f

In [ ]:
# downloading our dataset
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2025-08-27 17:13:05--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.1s    

2025-08-27 17:13:06 (8.93 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [5]:
with open('./input.txt','r',encoding='utf-8') as f:
    text = f.read()
print("lenth of dataset is:",len(text))

lenth of dataset is: 1115394


In [14]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [21]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [44]:
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[n] for n in l])

In [41]:
print(encode('hii there'))

[46, 47, 47, 1, 58, 46, 43, 56, 43]


In [45]:
decode(encode('hii there'))

'hii there'

In [48]:
data = torch.tensor(encode(text),dtype=torch.long)
print(data.shape,data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [54]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [53]:
len(val)/len(train) * 100

11.111177521830864

In [55]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [59]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f'When input is: {context}, The output is {target}')

When input is: tensor([18]), The output is 47
When input is: tensor([18, 47]), The output is 56
When input is: tensor([18, 47, 56]), The output is 57
When input is: tensor([18, 47, 56, 57]), The output is 58
When input is: tensor([18, 47, 56, 57, 58]), The output is 1
When input is: tensor([18, 47, 56, 57, 58,  1]), The output is 15
When input is: tensor([18, 47, 56, 57, 58,  1, 15]), The output is 47
When input is: tensor([18, 47, 56, 57, 58,  1, 15, 47]), The output is 58


In [84]:
torch.manual_seed(42)
batch_size = 4
block_size = 8

def get_batch(split:str):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data)-block_size,(batch_size,))
    x = torch.stack([data[i:i+block_size]for i in ix])
    y = torch.stack([data[i+1:i+block_size+1]for i in ix])
    return x,y

xb,yb = get_batch('train')
print("X's",xb.shape)
print("Y's",yb.shape)

X's torch.Size([4, 8])
Y's torch.Size([4, 8])


In [ ]:
import torch.nn as nn

class BigramLanguageModel(nn.Module):
    
    def __init__(self,vocab_size:int) -> None:
        super().__init__()
        # we embedd the tokens
        self.token_embedding_table = nn.Embedding(vocab_size,vocab_size)
    
    def forward(self,idx,targets=None):
        # we grab the corresponding embeddings
        logits = self.token_embedding_table(idx) # (B,T,C)
        
        if targets is None:
            loss = None
        else:
            B,T,C = logits.shape
            logits = logits.view(B*T,C)
            targets = targets.view(B*T)

            loss = nn.functional.cross_entropy(logits,targets)
        
        return logits,loss
    
    def generate(self,idx,max_tokens):
        for _ in range(max_tokens):
            logits,loss = self(idx)
            logits = logits[:,-1,:] # only get the last logit
            probs = nn.functional.softmax(logits,1)
            idnext = torch.multinomial(probs,num_samples=1)
            idx = torch.cat((idx,idnext),dim=1)
        return idx



m = BigramLanguageModel(vocab_size)
out,loss = m(xb,yb)
print(out.shape)
print(f'{loss.item():.4f}')

torch.Size([32, 65])
4.3425


In [ ]:
optimizer = torch.optim.AdamW(m.parameters(),lr = 1e-3)

In [200]:
batch_size = 32

for steps in range(10000):

    xb,yb = get_batch('train')

    logits,loss = m(xb,yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

2.476790189743042


In [202]:
ix = torch.zeros((1,1),dtype=torch.long)
print((decode(m.generate(ix,500)[0].tolist())))


Tonde, je ubrcept cellil my ms.
Lilubr a ng urpounathe thouput Corba ye, ct heakinjone pathad te ss s thaileest,
CIO:
DI fal sly iltes sloth tho eainted Sotls m. brs h of t t Prectee cand wive ie. heng ntce ell ltisthe'st hag?
IVIUS:

Thtirk hed kndanrtou ay,

Whis.
S:
BROrait y oate whthes war mirdearwhe bano fends I faigin; f whenonlecesaveanghey ispugs win ar.

MESo, the s CAno.
Medse avee w hive, f juisher ixpowreyo liche prailendspathimy n gsomen se, tyo hty y ay beint O sis, r deep'sirtu, 


In [134]:
idx = torch.randint(0,100,(2,5))
idnext = torch.tensor([[2,1],
                       [2,2]])
idx = torch.cat((idx,idnext),dim=1)
idx

tensor([[62, 95, 97, 81, 56,  2,  1],
        [54, 80, 86, 30,  7,  2,  2]])

In [177]:
tokens = torch.randint(0,65,(4,8))
embedding_matrix = torch.randn(65,65)
final_matrix = nn.functional.embedding(tokens, embedding_matrix)
final_matrix.shape

torch.Size([4, 8, 65])